In [ ]:
------------LOGIN---------
snowsql -a "YOURINSTANCE" -u "YOURUSER"

In [ ]:
------------CREATE USER----------
USE ROLE ACCOUNTADMIN;
CREATE SECURITY INTEGRATION SNOWSERVICES_INGRESS_OAUTH
  TYPE=oauth
  OAUTH_CLIENT=snowservices_ingress
  ENABLED=true;

CREATE USER weaviate_user PASSWORD='weaviate123' DEFAULT_ROLE = WEAVIATE_ROLE DEFAULT_SECONDARY_ROLES = ('ALL') MUST_CHANGE_PASSWORD = FALSE;

GRANT ROLE WEAVIATE_ROLE TO USER weaviate_user;

ALTER USER weaviate_user SET DEFAULT_ROLE = WEAVIATE_ROLE;


In [ ]:
------CREATE PRODUCT REVIEWS DATABASE -----
USE ROLE SYSADMIN;

CREATE OR REPLACE WAREHOUSE WEAVIATE_WAREHOUSE WITH
  WAREHOUSE_SIZE='X-SMALL'
  AUTO_SUSPEND = 180
  AUTO_RESUME = true
  INITIALLY_SUSPENDED=false;

CREATE DATABASE IF NOT EXISTS WEAVIATE_PRODUCT_REVIEWS;

GRANT all on database WEAVIATE_PRODUCT_REVIEWS to role WEAVIATE_ROLE;

USE DATABASE WEAVIATE_PRODUCT_REVIEWS;

In [ ]:
--------CREATE FILE FORMAT  -----

create or replace file format my_csv_format 
  type = csv 
  field_delimiter = ',' 
  skip_header = 1 
  null_if = ('NULL', 'null') 
  FIELD_OPTIONALLY_ENCLOSED_BY='"'
  empty_field_as_null = true;

-----CREATE STAGES--------

CREATE OR REPLACE STAGE REVIEW_DATA ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');
CREATE OR REPLACE STAGE YAML_STAGE;
CREATE OR REPLACE STAGE DATA ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');
CREATE OR REPLACE STAGE FILES ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

grant all PRIVILEGES on stage REVIEW_DATA to WEAVIATE_ROLE;
grant all PRIVILEGES on schema PUBLIC to WEAVIATE_ROLE;
grant all on schema PUBLIC to role WEAVIATE_ROLE;

In [ ]:
--------CREATE TABLES------------

CREATE OR REPLACE TABLE 
    PRODUCT_REVIEWS (REVIEWERID varchar, ASIN varchar, 
    REVIEWERNAME varchar, HELPFUL varchar, 
    REVIEWTEXT varchar, OVERALL varchar, 
    SUMMARY varchar, UNIXREVIEWTIME varchar, REVIEWTIME varchar);

CREATE OR REPLACE TABLE 
    PRODUCTS (ASIN varchar,
    NAME varchar, 
    REVIEW_SUMMARY varchar,
    DESCRIPTION varchar, 
    FEATURES varchar);



In [ ]:
----------PUT DATA FILE INTO STAGE ------------
PUT file:///<PATH>/Data/Musical_instruments_reviews.csv @REVIEW_DATA overwrite=true;

------------COPY DATA INTO SNOWFLAKE TABLES------------
COPY INTO PRODUCT_REVIEWS FROM @REVIEW_DATA FILE_FORMAT = (format_name = 'my_csv_format' , error_on_column_count_mismatch=false) 
  pattern = '.*Musical_instruments_reviews.csv.gz' on_error = 'skip_file';

--------Confirm data is there----------
SELECT REVIEWERID FROM PRODUCT_REVIEWS; 

-------GRANT PERMISSION TO USER --------
GRANT SELECT ON TABLE PRODUCT_REVIEWS TO ROLE WEAVIATE_ROLE;
GRANT ALL on TABLE PRODUCTS TO ROLE WEAVIATE_ROLE;


Update Spec files
Update the image URL with both the account and the database format_name

get the accountname-orgname value from snowflake
Account-Org 


Navigate to folder OllamaDocker in repo

In [ ]:

--Create an image repo
CREATE OR REPLACE IMAGE REPOSITORY WEAVIATE_PRODUCT_REVIEWS.PUBLIC.WEAVIATE_REPO;

--upload yaml files
PUT file:///<PATH>spec-jupyter.yaml @yaml_stage overwrite=true auto_compress=false;
PUT file:///<PATH>spec-text2vec.yaml @yaml_stage overwrite=true auto_compress=false;
PUT file:///<PATH>spec-weaviate.yaml @yaml_stage overwrite=true auto_compress=false;
PUT file:///<PATH>spec-ollama.yaml @yaml_stage overwrite=true auto_compress=false;


In [ ]:
#Login to snowflake with docker first
docker login <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com -u <USER>

Grab a copy of the Ollama repo
https://github.com/jmorganca/ollama


Copy over the contents of the cookbooks/Ollama/OllamaDocker folder to the root of the Ollama repo

In [ ]:
#Build ollama
docker build --rm --platform linux/amd64 -t ollama .
#Tag
docker tag ollama <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/ollama
#Push
docker push <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/ollama


In [ ]:

#----weaviate------
docker build --rm --platform linux/amd64 -t weaviate -f weaviate.Dockerfile . 
docker tag weaviate <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/weaviate
docker push <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/weaviate

#----text2vec------
docker build --rm --platform linux/amd64 -t text2vec -f text2vec.Dockerfile . 
docker tag text2vec <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/text2vec
docker push <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/text2vec

#--------Jupyter-------
docker build --rm --platform linux/amd64 -t jupyter -f jupyter.Dockerfile . 
docker tag jupyter <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/jupyter
docker push <SNOWACCOUNT-SNOWORG>.registry.snowflakecomputing.com/weaviate_product_reviews/public/weaviate_repo/jupyter


In [ ]:
-----------Create Compute Pools--------
CREATE COMPUTE POOL OLLAMA_POOL
MIN_NODES = 1
MAX_NODES = 1
INSTANCE_FAMILY = GPU_NV_M;

DESCRIBE COMPUTE POOL OLLAMA_POOL;


The compute pool takes 3-4 mins to start

In [ ]:
---------Create service------------

  CREATE SERVICE  OLLAMA
  IN COMPUTE POOL OLLAMA_POOL
  FROM @YAML_STAGE
  SPEC='spec-ollama.yaml'
  MIN_INSTANCES=1
  MAX_INSTANCES=1;

show services;

The service takes about 5+ mins to start

In [ ]:
GRANT USAGE ON SERVICE OLLAMA TO ROLE WEAVIATE_ROLE;
--Get the logs
CALL SYSTEM$GET_SERVICE_LOGS('WEAVIATE_PRODUCT_REVIEWS.PUBLIC.OLLAMA', '0', 'ollama');
--Check the status
CALL SYSTEM$GET_SERVICE_STATUS('WEAVIATE_PRODUCT_REVIEWS.PUBLIC.OLLAMA', 100); 

SHOW ENDPOINTS IN SERVICE OLLAMA;



Grab the endpoint for jupyter and navigate to it in your browser

login with weaviate_user and weaviate123
Note - this will take a few more mins to load the page 

Grab the key for jupyter from the logs using this command:
CALL SYSTEM$GET_SERVICE_STATUS('WEAVIATE_PRODUCT_REVIEWS.PUBLIC.OLLAMA', 100); 

Login to jupyter by getting the endpoint with this command:
SHOW ENDPOINTS IN SERVICE OLLAMA;

Use key to login

Once into Jupyter, create a console session
start ollama
/bin/ollama serve

launch the model
ollama run mistral

ollama should be running now

In [ ]:
---Create the rest of the services------
CREATE COMPUTE POOL IF NOT EXISTS WEAVIATE_COMPUTE_POOL
  MIN_NODES = 1
  MAX_NODES = 1
  INSTANCE_FAMILY = CPU_X64_S
  AUTO_RESUME = true;

CREATE COMPUTE POOL IF NOT EXISTS TEXT2VEC_COMPUTE_POOL
  MIN_NODES = 1
  MAX_NODES = 1
  INSTANCE_FAMILY = GPU_NV_S
  AUTO_RESUME = true;

CREATE COMPUTE POOL IF NOT EXISTS JUPYTER_COMPUTE_POOL
  MIN_NODES = 1
  MAX_NODES = 1
  INSTANCE_FAMILY = CPU_X64_S
  AUTO_RESUME = true;


CREATE SERVICE WEAVIATE
  IN COMPUTE POOL WEAVIATE_COMPUTE_POOL 
  FROM @YAML_STAGE
  SPEC='spec-weaviate.yaml'
  MIN_INSTANCES=1
  MAX_INSTANCES=1;

CREATE SERVICE JUPYTER
  IN COMPUTE POOL JUPYTER_COMPUTE_POOL 
  FROM @YAML_STAGE
  SPEC='spec-jupyter.yaml'
  MIN_INSTANCES=1
  MAX_INSTANCES=1;

CREATE SERVICE TEXT2VEC
  IN COMPUTE POOL TEXT2VEC_COMPUTE_POOL 
  FROM @YAML_STAGE
  SPEC='spec-text2vec.yaml'
  MIN_INSTANCES=1
  MAX_INSTANCES=1;


In [ ]:
GRANT USAGE ON SERVICE JUPYTER TO ROLE WEAVIATE_ROLE;
GRANT USAGE ON SERVICE WEAVIATE TO ROLE WEAVIATE_ROLE;
GRANT USAGE ON SERVICE TEXT2VEC TO ROLE WEAVIATE_ROLE;

CALL SYSTEM$GET_SERVICE_LOGS('WEAVIATE_PRODUCT_REVIEWS.PUBLIC.JUPYTER', '0', 'jupyter');
SHOW ENDPOINTS IN SERVICE jupyter;

CALL SYSTEM$GET_SERVICE_LOGS('WEAVIATE_PRODUCT_REVIEWS.PUBLIC.WEAVIATE', '0', 'weaviate');


In [ ]:
-------Update Permissions-----------
GRANT USAGE ON SERVICE JUPYTER TO ROLE WEAVIATE_ROLE;
GRANT USAGE ON SERVICE WEAVIATE TO ROLE WEAVIATE_ROLE;
GRANT USAGE ON SERVICE TEXT2VEC TO ROLE WEAVIATE_ROLE;
GRANT USAGE ON SERVICE MISTRAL TO ROLE WEAVIATE_ROLE;
GRANT USAGE on warehouse WEAVIATE_WAREHOUSE to role WEAVIATE_ROLE;

In [ ]:
-----temporarily pause instance ----------
ALTER COMPUTE POOL MISTRAL_POOL SUSPEND;
ALTER SERVICE MINSTRAL SUSPEND;

-----------Cleanup stuff that costs money---------------
USE ROLE ACCOUNTADMIN;

DROP SERVICE WEAVIATE;
DROP SERVICE JUPYTER;
DROP SERVICE TEXT2VEC;
DROP SERVICE OLLAMA;

DROP COMPUTE POOL WEAVIATE_COMPUTE_POOL;
DROP COMPUTE POOL JUPYTER_COMPUTE_POOL;
DROP COMPUTE POOL TEXT2VEC_COMPUTE_POOL;
DROP COMPUTE POOL OLLAMA_POOL;

In [ ]:
----Really clean up everything------

DROP IMAGE REPOSITORY WEAVIATE_PRODUCT_REVIEWS.PUBLIC.WEAVIATE_REPO;

DROP DATABASE WEAVIATE_PRODUCT_REVIEWS.PUBLIC;

DROP WAREHOUSE WEAVIATE_WAREHOUSE;

DROP USER weaviate_user;

DROP ROLE WEAVIATE_ROLE;

DROP SECURITY INTEGRATION SNOWSERVICES_INGRESS_OAUTH;